In [ ]:
!pip install xgboost


In [94]:
import numpy as np
import pandas as pd

In [95]:
df = pd.read_csv('ecopackai_frozen_materials.csv')
df.head()

,material_id,material_name,strength,weight_capacity,cost,biodegradibility_score,co2_score,recyclability_percentage
0,1,Thin Paper Wrap,1,5,2,8,3,85
1,2,Lightweight Kraft Paper,1,6,2,8,3,88
2,3,Paper Cushioning Sheets,1,7,3,8,3,90
3,4,Cellulose Padding,1,6,3,8,3,80
4,5,Compostable Film,1,5,4,9,3,45


In [96]:
x = df[['strength','weight_capacity','biodegradibility_score','recyclability_percentage']]
y_cost = df['cost'].values
y_co2 = df['co2_score'].values

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [98]:
x_train,x_test,y_train_cost,y_test_cost,y_train_co2,y_test_co2 = train_test_split(x,y_cost,y_co2,test_size=0.2,random_state=42)
y_train_cost.shape

(40,)

# scaling

In [99]:
scalar = StandardScaler()
x_train_scaled = scalar.fit_transform(x_train)
x_test_scaled = scalar.transform(x_test)
x_train_scaled.shape


(40, 4)

In [100]:
print(np.mean(x_train_scaled,axis=0))
print(np.std(x_train_scaled,axis=0))

[ 4.44089210e-17  6.66133815e-17 -1.22124533e-16  3.99680289e-16]
[1. 1. 1. 1.]


# cost prediction

In [101]:
from sklearn.ensemble import RandomForestRegressor
rf_cost=RandomForestRegressor(n_estimators=100,random_state=42)
rf_cost.fit(x_train_scaled,y_train_cost)

RandomForestRegressor(random_state=42)

In [102]:
y_cost_pred = rf_cost.predict(x_test_scaled)

In [103]:
y_cost_pred

array([5.03      , 4.08      , 6.95      , 4.75233333, 5.19      ,
       3.693     , 5.78      , 6.86      , 6.94      , 5.75      ])

In [104]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test_cost, y_cost_pred)
rmse = np.sqrt(mean_squared_error(y_test_cost, y_cost_pred))
r2 = r2_score(y_test_cost, y_cost_pred)
print("MAE:",mae)
print("RMSE:",rmse)
print("R2:",r2)
        

MAE: 0.7805333333333331
RMSE: 0.8237872567868745
R2: 0.7230100226757372


# co2 prediction

In [105]:
from xgboost import XGBRegressor
xgb_co2 = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_co2.fit(x_train_scaled,y_train_co2)
y_co2_pred = xgb_co2.predict(x_test_scaled)

In [106]:
y_co2_pred

array([3.0001655, 3.0001655, 5.9987063, 2.5007472, 3.0002441, 2.499962 ,
       6.993316 , 6.993316 , 5.9987063, 4.000346 ], dtype=float32)

In [107]:
xgb_co2.predict(x_train_scaled)

array([2.9999359, 2.5007472, 5.999341 , 3.9997182, 3.0001872, 3.0001655,
       3.0001655, 3.0000548, 3.9995856, 3.0001655, 2.9999359, 3.9997752,
       6.993316 , 3.9997752, 5.9987063, 3.0001655, 2.5007472, 6.000251 ,
       5.9987063, 3.9996965, 4.9954567, 2.0022626, 5.999341 , 3.0001655,
       4.000346 , 3.0001655, 2.9999359, 3.9997752, 5.9973526, 3.0001655,
       2.0022626, 3.0001872, 4.9933395, 3.0001655, 3.0001564, 3.0001655,
       3.0001655, 2.0106504, 7.973043 , 3.9996965], dtype=float32)

In [108]:
mae = mean_absolute_error(y_test_co2, y_co2_pred)
rmse = np.sqrt(mean_squared_error(y_test_co2, y_co2_pred))
r2 = r2_score(y_test_co2, y_co2_pred)
print("MAE:",mae)
print("RMSE:",rmse)
print("R2:",r2)

MAE: 0.8015679121017456
RMSE: 1.3233043923484327
R2: 0.6469486951828003


# normalizing cost and co2

In [109]:
df['cost_score_norm'] = 1 - (
    (df['cost'] - df['cost'].min()) /
    (df['cost'].max() - df['cost'].min())
)

df['co2_score_norm'] = 1 - (
    (df['co2_score'] - df['co2_score'].min()) /
    (df['co2_score'].max() - df['co2_score'].min())
)


# suitability score

In [110]:
df['suitability_score'] = (
    0.4 * (df['strength'] / 3) +
    0.3 * (df['recyclability_percentage'] / 100) +
    0.3 * (df['biodegradibility_score'] / 10)
)
#4 3 3

# final score

In [111]:
df['final_score'] = (
    0.4 * df['cost_score_norm'] +
    0.4 * df['co2_score_norm'] +
    0.2 * df['suitability_score']
)


In [112]:
df_sorted = df.sort_values(by='final_score', ascending=False)
df_sorted[['material_name', 'final_score']].head(10)


,material_name,final_score
1,Lightweight Kraft Paper,0.870324
0,Thin Paper Wrap,0.868524
49,Light Paper Sleeve,0.868524
7,Kraft Paper,0.832724
11,Palm Leaf Packaging,0.823200
2,Paper Cushioning Sheets,0.804857
48,Cotton Fiber Wrap,0.801857
3,Cellulose Padding,0.798857
46,Thin Jute Fabric,0.792857
14,Molded Pulp,0.791000


In [113]:
import joblib

In [114]:
joblib.dump(rf_cost,"D:\ecopack_ai\models\cost_model.pkl")
joblib.dump(xgb_co2,"D:\ecopack_ai\models\co2_model.pkl")



['D:\\ecopack_ai\\models\\co2_model.pkl']

In [115]:
joblib.dump(scalar,r"D:\ecopack_ai\models\feature_scaler.pkl")

['D:\\ecopack_ai\\models\\feature_scaler.pkl']